# 4. 사용자의 평가경향을 고려한 CF

In [31]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 읽어오기(user, item, data)

u_cols = ['user_id','age','sex','occupation', 'zip_code'] # 사용자아이디, 나이, 성별, 직업, 우편번호
users = pd.read_csv('data/u.user', 
                    sep='|', 
                    names=u_cols, 
                    encoding = 'latin-1')
users = users.set_index('user_id')


i_cols = ['movie_id','title', 'release date', 'video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation',
          'Children\'s','Comedy','Crime', 'Documentary','Drama', 
          'Fantasy','Film-Noir','Horror','Musical','Mystery','Romance',
          'Sci-Fi','Thriller','War','Western']
movies = pd.read_csv('data/u.item', 
                     sep='|', 
                     names=i_cols,
                     encoding = 'latin-1')
movies = movies.set_index('movie_id')


r_cols = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv('data/u.data', 
                     sep='\t', 
                     names=r_cols,
                     encoding = 'latin-1')




In [32]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred) :
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))


In [33]:
def score(model, neighbor_size=0) :
    #테스트 데이터의 user_id와 movie_id간 pair를 맞춰 튜플형원소 리스트데이터를 만듬
    id_pairs = zip(x_test['user_id'], x_test['movie_id']) # 튜플 형태로 만들어 줌
    
    #모든 사용자-영화 짝에 대해서 주어진 예측모델에 의한 예측값 계산 및 리스트형 데이터 생성
    y_pred = np.array([model(user, movie,neighbor_size) for (user, movie) in id_pairs])
    
    #실제 평점값
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)        

In [34]:
# 데이터 셋 만들기
x = ratings.copy()
y = ratings['user_id']


x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.25,
                                                    stratify=y) #계층화 추출
rating_matrix = x_train.pivot(index='user_id', 
                              columns='movie_id',
                              values='rating')


In [35]:
# 코사인 유사도 계산
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)


user_similarity = pd.DataFrame(user_similarity,
                               index = rating_matrix.index,
                               columns = rating_matrix.index)
print(user_similarity)


user_id       1         2         3         4         5         6         7    \
user_id                                                                         
1        1.000000  0.116919  0.047989  0.067611  0.305415  0.343038  0.369981   
2        0.116919  1.000000  0.072933  0.131519  0.032758  0.203473  0.065847   
3        0.047989  0.072933  1.000000  0.195390  0.027846  0.063695  0.066068   
4        0.067611  0.131519  0.195390  1.000000  0.041446  0.080036  0.067818   
5        0.305415  0.032758  0.027846  0.041446  1.000000  0.173378  0.243990   
...           ...       ...       ...       ...       ...       ...       ...   
939      0.097422  0.250733  0.035688  0.040704  0.066928  0.102331  0.080348   
940      0.251135  0.176157  0.132746  0.176143  0.214383  0.267901  0.244684   
941      0.107922  0.139638  0.108527  0.136157  0.147165  0.124721  0.062734   
942      0.154504  0.120087  0.096442  0.122947  0.126829  0.261880  0.210407   
943      0.334440  0.078667 

In [36]:
# 각 사용자의 평균을 구한 후 평균과의 차이를 구함






In [37]:
#type(rating_matrix)

In [38]:
# 사용자 평가 경향을 고려한 함수
def CF_knn_bias(user_id, movie_id, neighbor_size=0) :##### Neighbor size를 정해서 예측치를 계산하는 함수

  # train/test set의 분할에 따라 rating_matrix에 해당 영화가 있는지 확인
  if movie_id in rating_matrix.columns:
    # 주어진 사용자(user_id)와 다른 사용자의 유사도 추출
    sim_scores = user_similarity[user_id].copy()
    # 주어진 영화(movie_id)와 다른 사용자의 유사도 추출
    movie_ratings = rating_matrix[movie_id].copy()
    # 주어진 영화에 대해서 펴아를 하지 않은 사용자를 가중평균계산에서 제외하기 위한 인덱스 추출
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    # 주어진 영화에 대해서 평가를 하지 않은 사람 제외
    movie_ratings = movie_ratings.dropna()
    # 주어진 영화를 평가하지 않은 사용자와의 유사도를 제거, 가중 평균 계산할 때 필요가 없기 때문
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0: # neighbor_size가 지정되지 않은 경우
      mean_rating = np.dot(sim_scores, movie_ratings)/sim_scores.sum()
    else : # neighbor_size가 지정된 경우
      if len(sim_scores)>1 :
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        mean_rating = np.dot(sim_scores, movie_ratings)/sim_scores.sum()
      else :
        mean_rating = 3.0
  else : # train/test set의 분할에 따라 rating_matrix에 해당 영화가 없으면 기본값 3.0 예측치로 간주
    mean_rating = 3.0
    
  return mean_rating

#score(CF_knn_bias, 10)
                                 